<a href="https://colab.research.google.com/github/darknet-doll/notebook_error_logger/blob/main/Sample_Notebook_Impact_Logger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [15]:
# Import from package from error log package from github.
# Ensure error_logs.db is not in current runtime.
# !pip install -e . # For local dev only.
!pip install git+https://github.com/darknet-doll/notebook_error_logger.git --upgrade
from notebook_error_logger import start_logger
from notebook_error_logger.airtable_sync import write_airtable, read_airtable

# Import python packagess.
import sqlite3, pandas as pd

# Credentials

# Local dev only.
!pip install python-dotenv
from dotenv import load_dotenv
import os
load_dotenv()

if os.getenv("LOCAL_DEV") == "true":
    print("Local dev mode: ON")
    AIRTABLE_TOKEN = os.getenv("AIRTABLE_TOKEN")
    AIRTABLE_BASE_ID = os.getenv("AIRTABLE_BASE_ID")
else:
    # Colab
    from google.colab import userdata
    AIRTABLE_TOKEN = userdata.get("AIRTABLE_TOKEN")
    AIRTABLE_BASE_ID = userdata.get("AIRTABLE_BASE_ID")

TABLE_NAME = "Errors"

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/darknet-doll/notebook_error_logger.git to /private/var/folders/sq/3cdvc0vx48xcfjxg6c93b61w0000gn/T/pip-req-build-tesuluy0
  Running command git clone --filter=blob:none --quiet https://github.com/darknet-doll/notebook_error_logger.git /private/var/folders/sq/3cdvc0vx48xcfjxg6c93b61w0000gn/T/pip-req-build-tesuluy0
  Resolved https://github.com/darknet-doll/notebook_error_logger.git to commit a47cd5b6cdbf671901eedf2c478e92cdcf06e9b3
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'notebook-error-logger' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'notebook-error-logger'

# Log Error

In [16]:
# Start logging errors by creating project_name field.
logger = start_logger("Sample Notebook")

In [17]:
# Testing errors.

1 / 0 # trigger an error.

# a = er ror_syntax # undefined variable cannot be assigned.

ZeroDivisionError: division by zero

In [18]:
# Reviewing errors logged in error_logs.db dataframe for this project in SQLlite.
project_log_df = pd.read_sql_query("SELECT * FROM errors", sqlite3.connect("error_logs.db"))
project_log_df

,id,project_type,project_name,error_type,date
0,1,data science notebook,Sample Notebook,ZeroDivisionError,2025-11-17


# Airtable Connection

In [19]:
# Write local project_log_df to airtable.
write_airtable(project_log_df,
               airtable_token=AIRTABLE_TOKEN,
               base_id=AIRTABLE_BASE_ID
               )


✅ All records uploaded successfully to Airtable.


In [20]:
# Read records from table.
# Validate read from link https://airtable.com/app8lLddOSQt2Fhaz/shrAvfz8PwokANwqR/tblcNUsattIK0IKun.
df_from_airtable = read_airtable(airtable_token=AIRTABLE_TOKEN,
                   base_id=AIRTABLE_BASE_ID)

# Review relevant fields.
df_from_airtable[[
    'count_id',
    'date',
    'lookup_project_type',
    'project_name',
    'error_type'
    ]].sort_values(by='count_id')

✅ Retrieved 12 records from Airtable.


,count_id,date,lookup_project_type,project_name,error_type
8,1,2025-10-28,[data science notebook],test_session,ZeroDivisionError
7,2,2025-10-28,[data science notebook],test_session,ZeroDivisionError
0,3,2025-11-12,[data science notebook],Sample Notebook,ZeroDivisionError
3,4,2025-11-13,[data science notebook],Sample Notebook,NameError
4,5,2025-11-13,[maker],Test Prototype,PrototypeError
1,6,2025-11-14,[data science notebook],Sample Notebook,ZeroDivisionError
5,7,2025-11-17,[data science notebook],Sample Notebook,ZeroDivisionError
9,8,2025-11-17,[data science notebook],Sample Notebook,ZeroDivisionError
2,9,2025-11-17,[data science notebook],Sample Notebook,ZeroDivisionError
10,10,2025-11-17,[data science notebook],Sample Notebook,ZeroDivisionError
